# Create Stripes

In [1]:
import svgwrite as svg
import math
import os

import xml.etree.ElementTree as ET

import pandas as pd
import math

## Settings

In [2]:
musicxml = '../sheet/crowns_and_tears.musicxml'
covidstats = 'COVID19Death_geoRegion.csv'
startdate = '2020-02-25'

## Convert MusicXML to Array

In [3]:
root = ET.parse(musicxml).getroot()
division = int(root.find('.//attributes/divisions').text)
beat = int(root.find('.//attributes/time/beats').text)

In [4]:
def note_to_strip_number(step, octave):
    step = step.upper()
    value = 0
    if step == 'C': value = 0
    elif step == 'D': value = 1
    elif step == 'E': value = 2
    elif step == 'F': value = 3
    elif step == 'G': value = 4
    elif step == 'A': value = 5
    elif step == 'B': value = 6
    elif step == 'C': value = 7
        
    if octave == 5: value += 7
    elif octave == 6: value += 14
    
    return value        
    

In [5]:
data = []
# Add empty bars
totalbars = len(root.find('part').findall('measure'))
for i in range(0, int(totalbars) * int(beat) * int(division)):
    data.append([])
    
# Loop Parts
for part in root.findall('part'):
    
    # Loop measures
    
    stopdurationcount = False
    
    for measure in part.findall('measure'):
        durationcount = 0
        
        measure_i = int(measure.get('number'))
        
        for note_i, note in enumerate(measure.findall('note')):
            
            # Is note (not rest)
            pos = (measure_i - 1) * beat * division + durationcount
            if(note.find('rest') is None):
                #data[pos].append(note.find('pitch/step').text)
                data[pos].append(note_to_strip_number(note.find('pitch/step').text, int(note.find('pitch/octave').text)))
            #else:
            #    data[pos].append("break")

            # Stop count if...
            nextmeasure = measure.find('note[%s]' % (note_i + 2)) # remember: elem 0 = xpath 1
            if (nextmeasure is not None) and (
                (nextmeasure.find('chord') is not None)
            ):
                stopdurationcount = True
            else:
                stopdurationcount = False
            
            if not stopdurationcount:
                durationcount += int(note.find('duration').text)
                
            
        #if(measure_i >= 7):
        #    break
    

## Settings for Stripes

In [6]:
tmp_folder = 'export'

mm_height_paper = 41
mm_height = 28
mm_width = 260#263
mm_line_height = 2
mm_note_radius = 1
mm_bar_width = 2.5
mm_padding_start = 0#4

direction_right_to_left = True
plot_covid_bottom = False

## Import Covid Stats

In [7]:
covid_stat_height = mm_height

df = pd.read_csv(covidstats, encoding='latin-1')
df = df[df.geoRegion != 'FL']
df = df[df.datum >= startdate]

df = df.groupby('datum').agg({'entries': 'sum'})

df = df.reset_index()

# Remove "Summer break"
df = df[(df.datum <= '2020-07-21') | (df.datum >= '2020-09-20')]

# Calc Notes per day
df['height'] = df['entries'].apply(lambda x: round(covid_stat_height / df['entries'].max() * x, 1))

len(df)

329

## Plot Stripes

In [8]:
#Factor for Scaling up SVG - it is easier debugging it this way
factor = 5

bars_per_row = math.floor(mm_width / mm_bar_width)

for row in range (0, math.ceil(len(data) * mm_bar_width / mm_width)):
    
    # Create SVG
    d = svg.Drawing(os.path.join(tmp_folder, 'row_%s.svg' % row), size=((mm_width + mm_padding_start) * factor, mm_height_paper * factor))
    
    #--------------------- Add Covid Graph
    
    transf = "translate(0, %s)" % ((mm_height_paper - mm_height) / 2 * factor)
    
    # Create Group
    g_graph = svg.container.Group(transform = transf)
    
    # Sample DataFrame
    df_samp = df[math.floor(bars_per_row * row / division): math.ceil(bars_per_row * row / division + (bars_per_row / division))]
    
    # Empty path
    if plot_covid_bottom:
        path = "M %s %s" % (mm_width * factor, covid_stat_height * factor)
    else:
        path = "M %s %s" % (mm_width * factor, 0)  
    
    i = 0
    for ibar, df_row in df_samp.iterrows():
        if plot_covid_bottom:
            x = (mm_width * factor) - (i * division) * mm_bar_width * factor
            y = (covid_stat_height - df_row['height']) * factor
        else:
            #x = (i * division) * mm_bar_width * factor
            x = (mm_width * factor) - (i * division) * mm_bar_width * factor
            y = (df_row['height']) * factor
        
        path = path + "L %s %s " % (x, y)
            
        i += 1
        
    # Plot from last point to edge of strip
    if plot_covid_bottom:
        path = path + "L %s %s L %s %s Z" % (0, y, 0, covid_stat_height * factor)
    else:
        last = math.ceil(bars_per_row * row / division + (bars_per_row / division))
        df_samp = df[last: last + 1]
        if len(df_samp) > 0:
            path = path + "L %s %s L %s %s Z" % (0, df_samp.iloc[0]['height'] * factor, 0, 0)

    g_graph.add(svg.path.Path(path, style="fill:#98cbeb;stroke:#2c8eca"))
    
    d.add(g_graph) 
    
    
    #--------------------- Add Music

    # Add Borders
    #d.add(svg.shapes.Rect((0, 0), ((mm_width + mm_padding_start) * factor, mm_height_paper * factor),  style="stroke:#7b7b7b;stroke-width:0.1; fill: none"))
    d.add(svg.shapes.Line((0, 0), ((mm_width + mm_padding_start) * factor,  0), style="stroke:#7b7b7b;stroke-width:0.1"))
    d.add(svg.shapes.Line((0,  mm_height_paper * factor), ((mm_width + mm_padding_start) * factor, mm_height_paper * factor), style="stroke:#7b7b7b;stroke-width:0.1"))

    #transf = "translate(0, %s)" % ((mm_height_paper - mm_height) / 2 * factor)
    g = svg.container.Group(transform = transf)

    # Draw Lines
    for i in range(0, 15):
        g.add(svg.shapes.Line((0, i * mm_line_height * factor), (mm_width * factor, i * mm_line_height * factor), style="stroke:#a1a1a1;stroke-width:1"))

    # Draw Music
    for ibar, bar in enumerate(data[bars_per_row * row: bars_per_row * row + bars_per_row]):
        for note in bar:
            if direction_right_to_left:
                x = (mm_width * factor) - ibar * mm_bar_width * factor
            else:
                x = ibar * mm_bar_width * factor
            #y = (14 - note) * mm_line_height * factor
            y = note * mm_line_height * factor
            g.add(svg.shapes.Circle((x, y), r = (mm_note_radius * factor), style="fill: #686868"))



    d.add(g)

    # Save SVG
    d.save()

## Create PDF (if needed)

In [9]:
from reportlab.pdfgen.canvas import Canvas
from reportlab.lib.pagesizes import A4, landscape
from reportlab.lib.units import mm
from reportlab.graphics import renderPDF, renderPM
from reportlab.platypus import SimpleDocTemplate
from svglib.svglib import svg2rlg

In [10]:
# Create PDF

padding_metween_rows = 0

pdf_row_height = (mm_height_paper + padding_metween_rows) * mm

my_canvas = Canvas(os.path.join(tmp_folder, 'test.pdf'), pagesize=landscape(A4))

for row in range(0, math.ceil(len(data) * mm_bar_width / mm_width)):
                
    drawing = svg2rlg(os.path.join(tmp_folder, 'row_%s.svg' % row))

    #Calc Scaling
    scale_x = mm / factor
    drawing.scale(scale_x, scale_x)


    renderPDF.draw(drawing, my_canvas, 50, A4[0] - (60 * mm) - (row % 4 * pdf_row_height))
    
    # New Page
    if row % 4 == 3:
        #break
        my_canvas.showPage()
        

my_canvas.save()